In [5]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import numpy.typing as npt
import pdal

import xarray as xr
import xarray.typing as xrt


from forest_structure_tools.z_metrics import z_metrics, z_metrics_xr

In [6]:
data_dir = Path("../data")
plots_dir = data_dir / "outputs" / "plots"
plots_lidar_dir = plots_dir / "lidar"

In [7]:
def read_plot(plot_id: str):
    file_path = plots_lidar_dir / f"{plot_id}.copc.laz"
    pl = pdal.Reader(str(file_path)).pipeline()
    pl.execute()
    return pl.arrays[0]

In [13]:
def points_to_ds(points: npt.NDArray) -> xr.Dataset:
    x = points['X']
    y = points['Y']
    z = points['Z']
    rn = points['ReturnNumber']
    nor = points['NumberOfReturns']
    i = points['Intensity']

    return xr.Dataset(
        data_vars={
            "x": ("point_idx", x),
            "y": ("point_idx", y),
            "z": ("point_idx", z),
            "rn": ("point_idx", rn),
            "nor": ("point_idx", nor),
            "i": ("point_idx", i),
        }
    )


In [14]:
points = read_plot("AGG_O_01_P1")
points_ds = points_to_ds(points)

In [15]:
points_ds

<xarray.Dataset> Size: 7MB
Dimensions:  (point_idx: 252716)
Dimensions without coordinates: point_idx
Data variables:
    x        (point_idx) float64 2MB 4.63e+05 4.63e+05 ... 4.63e+05 4.63e+05
    y        (point_idx) float64 2MB 5.26e+06 5.26e+06 ... 5.26e+06 5.26e+06
    z        (point_idx) float64 2MB 0.0 0.0 0.0 0.0 ... 2.912 0.622 0.574 0.314
    rn       (point_idx) uint8 253kB 1 1 1 1 1 1 1 1 1 1 ... 1 2 1 1 1 2 1 1 1 1
    nor      (point_idx) uint8 253kB 1 1 1 1 1 1 1 1 1 1 ... 2 2 1 2 2 3 3 1 1 1
    i        (point_idx) uint16 505kB 31515 31348 31561 ... 31620 31387 31892

In [42]:
def combined_metrics(
    points_ds: xr.Dataset,
    xy_bin_size: float | None = None,
    z_bin_size: float | None = None,
    include_basic_metrics=True,
    percentiles=None,
    percentages=None,
    k: float = 1,
):
    x = points_ds["x"]
    y = points_ds["y"]
    z = points_ds["z"]

    if xy_bin_size is None:
        metrics = z_metrics_xr(
            z,
            include_basic_metrics=include_basic_metrics,
            percentiles=percentiles,
            percentages=percentages,
            k=k,
            zbin_size=z_bin_size,
        )
    else:
        x_bins = np.arange(x.min(), x.max() + xy_bin_size, xy_bin_size)
        y_bins = np.arange(y.min(), y.max() + xy_bin_size, xy_bin_size)

        x_bin_grouper = xr.groupers.BinGrouper(
            bins=x_bins, labels=x_bins[:-1], include_lowest=True
        )
        y_bin_grouper = xr.groupers.BinGrouper(
            bins=y_bins, labels=y_bins[:-1], include_lowest=True
        )

        xy_grouping = points_ds.groupby(x=x_bin_grouper, y=y_bin_grouper)
        xy_metrics = xy_grouping.map(
            lambda ds: z_metrics_xr(
                ds["z"],
                include_basic_metrics=include_basic_metrics,
                percentiles=percentiles,
                percentages=percentages,
                k=k,
                zbin_size=z_bin_size,
            )
        )

        metrics = xy_metrics.rename({ "x_bins": "x", "y_bins": "y"})

    metrics.attrs["xy_bin_size"] = xy_bin_size
    metrics.attrs["z_bin_size"] = z_bin_size
    metrics.attrs["k"] = k

    return metrics

In [58]:
gridded_metrics = combined_metrics(points_ds, xy_bin_size=5, z_bin_size=1)
plot_metrics = combined_metrics(points_ds, z_bin_size = 1)

In [53]:
gridded_metrics.mean(dim=['x', 'y'])

<xarray.Dataset> Size: 1kB
Dimensions:       (z: 18)
Coordinates:
  * z             (z) float64 144B 0.0 1.0 2.0 3.0 4.0 ... 14.0 15.0 16.0 17.0
Data variables: (12/15)
    max           float64 8B 13.1
    min           float64 8B 0.0
    range         float64 8B 13.1
    mean          float64 8B 4.081
    median        float64 8B 4.048
    sd            float64 8B 3.753
    ...            ...
    zbin_inside%  (z) float64 144B 24.66 19.1 2.963 ... 0.2335 0.1048 0.05403
    zbin_entries  (z) float64 144B 1.399e+03 2.635e+03 ... 9.486e+03 9.806e+03
    zbin_exits    (z) float64 144B nan 1.435e+03 2.635e+03 ... 9.476e+03 9.8e+03
    zbin_ppi      (z) float64 144B nan 0.5332 0.9321 ... 0.9977 0.999 0.9995
    zbin_vad      (z) float64 144B nan 0.6831 0.07117 ... 0.00105 0.0005405
    vai           float64 8B 1.515